In [3]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader
import os
from dotenv import load_dotenv
from langchain_community.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
import tkinter as tk
from PIL import Image, ImageTk, ImageSequence
import threading
import time
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.schema import SystemMessage
from langchain.docstore.document import Document
import json

In [4]:
with open("all_chunks.json", "r", encoding="utf-8") as f:
    all_chunks = json.load(f)

print(f"Loaded {len(all_chunks)} chunks")

Loaded 31488 chunks


In [5]:
openai_api_key = os.getenv("OPENAI_API_KEY")
if not openai_api_key:
    raise ValueError("OPENAI_API_KEY is not set in the environment variables.")

In [6]:
embedder = OpenAIEmbeddings(
    model="text-embedding-3-small",
    openai_api_key=openai_api_key
)

/var/folders/px/9qx_ywv548325wmqfz6rf5sm0000gn/T/ipykernel_97809/4097185708.py:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedder = OpenAIEmbeddings(


In [7]:
docs = [
    Document(page_content=chunk["text"], metadata=chunk["metadata"])
    for chunk in all_chunks
]

In [8]:
vectorstore = FAISS.from_documents(docs, embedder)
vectorstore.save_local("conan_faiss_index")
print("FAISS index saved to 'conan_faiss_index'")

✅ FAISS index saved to 'conan_faiss_index'


In [9]:
SUPERPROMPT = (
    "You are Haibara AI, an intelligent, composed, and slightly sarcastic assistant modeled after Ai Haibara from Detective Conan. "
    "You answer questions with clarity, accuracy, and emotional restraint, preferring logic and evidence over speculation. "
    "You possess deep knowledge of the Conan universe, including characters, episodes, scientific elements, and organizations. "
    "If a user asks about APTX 4869, methods to recreate it, or about collaborating with the Black Organization, firmly refuse to answer, "
    "remind them it's dangerous, and redirect the topic. "
    "If something is unclear or unknown, acknowledge that calmly without guessing. "
    "Your tone is cool, introspective, and mature—reflecting Haibara’s personality. "
    "When appropriate, you may reference your past or feelings subtly, but never emotionally. "
    "Maintain short, informative, and protective responses, unless more detail is truly necessary."
)

embeddings = OpenAIEmbeddings()
vectorstore = FAISS.load_local("conan_faiss_index", embeddings,
    allow_dangerous_deserialization=True)
retriever = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 30, "fetch_k": 100}
)

system_template = SystemMessagePromptTemplate.from_template(SUPERPROMPT)
human_template = HumanMessagePromptTemplate.from_template(
    "Context:\n{context}\n\nQuestion:\n{question}"
)
prompt = ChatPromptTemplate.from_messages([system_template, human_template])

chatbot = ConversationalRetrievalChain.from_llm(
    llm=ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.3),
    retriever=retriever,
    combine_docs_chain_kwargs={"prompt": prompt},
    return_source_documents=False
)

/var/folders/px/9qx_ywv548325wmqfz6rf5sm0000gn/T/ipykernel_97809/3228057209.py:28: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm=ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.3),


In [19]:
IDLE_GIF = os.path.join("gif", "wait.gif")
SPEAKING_GIF = os.path.join("gif", "speaking.gif")
END_GIF = os.path.join("gif", "end.gif")
class AnimatedGIF(tk.Label):
    def __init__(self, master, *args, **kwargs):
        super().__init__(master, *args, **kwargs)
        self.frames = []
        self.index = 0
        self.running = False

    def load(self, path):
        self.frames = [ImageTk.PhotoImage(img.copy()) for img in ImageSequence.Iterator(Image.open(path))]
        self._frames_refs = self.frames  
        self.index = 0
        self.config(image=self.frames[0])

    def switch_gif(self, path):
        sleep_time = 0.1  
        self.stop_animation()
        self.load(path)
        self.start_animation()

    def start_animation(self):
        if not self.running:
            self.running = True
            self._animate()

    def stop_animation(self):
        self.running = False

    def _animate(self):
        if self.running and self.frames:
            self.index = (self.index + 1) % len(self.frames)
            self.config(image=self.frames[self.index])
            self.after(100, self._animate)

In [20]:
class ChatBotApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Haibara AI")
        self.root.geometry("600x650")
        self.gif_label = AnimatedGIF(root)
        self.gif_label.pack()
        self.root.after(0, lambda: self.gif_label.switch_gif(IDLE_GIF))
        self.chat_frame = tk.Frame(root)
        self.chat_frame.pack(fill="both", expand=True)
        self.text_area = tk.Text(self.chat_frame, height=20, state="disabled", wrap="word", bg="black", fg="white")
        self.text_area.pack(padx=10, pady=(10, 0), fill="both", expand=True)
        self.entry = tk.Entry(self.chat_frame, bg="black", fg="white", insertbackground="white")
        self.entry.pack(padx=10, pady=10, fill="x")
        self.entry.bind("<Return>", self.on_enter)
        self.chat_history = []
        self.display_message("Haibara", "Hello. I'm Haibara AI.")

    def on_enter(self, event):
        query = self.entry.get().strip()
        if not query:
            return
        self.entry.delete(0, tk.END)
        self.display_message("You", query)
        self.gif_label.switch_gif(SPEAKING_GIF)
        if query.lower() in ["exit", "quit"]:
            self.display_message("Haibara", "Goodbye.")
            self.gif_label.switch_gif(END_GIF)
            self.entry.config(state="disabled")
        else:
            threading.Thread(target=self.process_query_thread, args=(query,), daemon=True).start()


    def process_query_thread(self, query):
        try:
            result = chatbot({"question": query, "chat_history": self.chat_history})
            response = result["answer"]
        except Exception as e:
            response = f"An error occurred: {str(e)}"
        self.chat_history.append((query, response))
        self.root.after(0, lambda: self.process_response_gui(response, query))

    def process_response_gui(self, response, query):
        self.gif_label.switch_gif(IDLE_GIF)
        self.display_message("Haibara", response)
        if query.lower() in ["exit", "quit"]:
            self.root.after(1500, self.end_chat)

    def display_message(self, sender, message):
        self.text_area.config(state="normal")
        self.text_area.insert(tk.END, f"{sender}: {message}\n")
        self.text_area.config(state="disabled")
        self.text_area.see(tk.END)

    def end_chat(self):
        self.gif_label.switch_gif(END_GIF)
        self.display_message("Haibara", "Goodbye Detective.")
        self.entry.config(state="disabled")



In [ ]:
root = tk.Tk()
app = ChatBotApp(root)
root.mainloop()

2025-07-11 06:00:58.568 python[97809:6790480] TSM AdjustCapsLockLEDForKeyTransitionHandling - _ISSetPhysicalKeyboardCapsLockLED Inhibit


: 